In [1]:
from keras.models import Sequential, load_model

import numpy as np

In [2]:
labelPath = 'data/labels.txt'
dataPath = 'data'

In [3]:
with open(labelPath, 'r') as f:
    lines = f.readlines()
    actions = [line.strip().split()[0] for line in lines]
    labels = [line.strip().split()[1] for line in lines]

In [4]:
data = np.concatenate([
    np.load(f'{dataPath}/seq_fist.npy')
], axis=0)

In [5]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

x_data = data[:, :, :-1]
labels = data[:, 0, -1]

y_data = to_categorical(labels, num_classes=len(actions))

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2022)

x_train.shape, y_train.shape

((258, 30, 99), (258, 3))

In [25]:
from keras.layers import LSTM, Dense, Reshape

base_model = load_model('models/transfer.h5')

# freeze some layers of original model
for layer in base_model.layers[:-2]:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 3)                 44163     
                                                                 
 dense_4 (Dense)             (None, 16)                64        
                                                                 
 dense_5 (Dense)             (None, 3)                 51        
                                                                 
Total params: 44,278
Trainable params: 2,294
Non-trainable params: 41,984
_________________________________________________________________


In [27]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/modeltest.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/200
9/9 [==============================] - ETA: 0s - loss: 0.0131 - acc: 1.0000
Epoch 1: val_acc improved from -inf to 1.00000, saving model to models\modeltest.h5
9/9 [==============================] - 1s 70ms/step - loss: 0.0131 - acc: 1.0000 - val_loss: 0.0124 - val_acc: 1.0000 - lr: 5.0000e-04
Epoch 2/200
8/9 [=========================>....] - ETA: 0s - loss: 0.0126 - acc: 1.0000
Epoch 2: val_acc did not improve from 1.00000
9/9 [==============================] - 1s 57ms/step - loss: 0.0128 - acc: 1.0000 - val_loss: 0.0121 - val_acc: 1.0000 - lr: 5.0000e-04
Epoch 3/200
9/9 [==============================] - ETA: 0s - loss: 0.0125 - acc: 1.0000
Epoch 3: val_acc did not improve from 1.00000
9/9 [==============================] - 0s 53ms/step - loss: 0.0125 - acc: 1.0000 - val_loss: 0.0118 - val_acc: 1.0000 - lr: 5.0000e-04
Epoch 4/200
9/9 [==============================] - ETA: 0s - loss: 0.0122 - acc: 1.0000
Epoch 4: val_acc did not improve from 1.00000
9/9 [================